In [2]:
import pandas as pd
import numpy as np

In [4]:
companies = pd.read_csv('./merged/new_companies.csv')
products = pd.read_csv('./merged/new_products.csv')


relationships_people = pd.read_csv('./merged/new_relationships_people.csv')
funding_rounds = pd.read_csv('./merged/new_funding_rounds.csv')
acquisitions = pd.read_csv('./merged/new_acquisitions.csv')
investments_funds = pd.read_csv('./merged/new_investments_funds.csv')
ipos = pd.read_csv('./merged/new_ipos.csv')


In [ ]:

product_condition = (
    (products['status'] != 'closed') &
    products['founded_at'].isna() & 
    products['closed_at'].isna() & 
    products['domain'].isna() &
    products['twitter_username'].isna() &
    products['overview'].isna() &
    products['tag_list'].isna()
)

products.loc[product_condition, 'status'] = 'closed' 


product_counts = products.groupby('parent_id').size()

# Merge the product counts with the companies DataFrame
companies = companies.merge(product_counts.rename('product_count'), left_on='id', right_index=True, how='left')

# Replace NaN values with 0 in product_count
companies['product_count'].fillna(0, inplace=True)

# Create new columns for closed and operating product counts
companies['closed_product_count'] = companies.apply(lambda row: row['product_count'] if row['status'] == 'closed' else 0, axis=1)
companies['operating_product_count'] = companies.apply(lambda row: row['product_count'] if row['status'] != 'closed' else 0, axis=1)

companies.head()

In [8]:
offices = pd.read_csv('./raw_data/offices.csv')

office_condition = (
    offices['description'].isna() &
    offices['address1'].isna() & 
    offices['address2'].isna() & 
    offices['city'].isna() &
    offices['zip_code'].isna() &
    offices['state_code'].isna()
)

offices.loc[office_condition, 'created_at'] = 'closed' 


office_counts = offices.groupby('object_id').size()

# Merge the product counts with the companies DataFrame
companies = companies.merge(office_counts.rename('office_count'), left_on='id', right_index=True, how='left')

# Replace NaN values with 0 in product_count
companies['office_count'].fillna(0, inplace=True)

# Create new columns for closed and operating product counts
companies['closed_office_count'] = companies.apply(lambda row: row['office_count'] if row['created_at'] == 'closed' else 0, axis=1)
companies['operating_office_count'] = companies.apply(lambda row: row['office_count'] if row['created_at'] != 'closed' else 0, axis=1)

companies.head()

/var/folders/6g/_ldjv_pd0k36jp2tc2jnbcl00000gn/T/ipykernel_47291/1744537565.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'closed' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  offices.loc[office_condition, 'created_at'] = 'closed'


,id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,domain,homepage_url,...,funding_rounds,funding_total_usd,created_at,updated_at,product_count,closed_product_count,operating_product_count,office_count,closed_office_count,operating_office_count
0,c:1,Company,NaN,Wetpaint,web,operating,2005-10-17,NaN,wetpaint-inc.com,http://wetpaint-inc.com,...,3,39750000.0,2007-05-25 06:51:27,2013-04-13 03:29:00,2.0,0.0,2.0,2.0,0,2.0
1,c:10,Company,NaN,Flektor,games_video,acquired,NaN,NaN,flektor.com,http://www.flektor.com,...,0,0.0,2007-05-31 21:11:51,2008-05-23 23:23:14,1.0,0.0,1.0,1.0,0,1.0
2,c:100,Company,NaN,There,games_video,acquired,NaN,NaN,there.com,http://www.there.com,...,0,0.0,2007-08-06 23:52:45,2013-11-04 02:09:48,0.0,0.0,0.0,1.0,0,1.0
3,c:10000,Company,NaN,MYWEBBO,network_hosting,operating,2008-07-26,NaN,mywebbo.com,http://www.mywebbo.com,...,0,0.0,2008-08-24 16:51:57,2008-09-06 14:19:18,0.0,0.0,0.0,0.0,0,0.0
4,c:10001,Company,NaN,THE Movie Streamer,games_video,operating,2008-07-26,NaN,themoviestreamer.com,http://themoviestreamer.com,...,0,0.0,2008-08-24 17:10:34,2008-09-06 14:19:18,0.0,0.0,0.0,0.0,0,0.0


In [12]:
# Assuming 'your_column_name' is the column you want to check
companies.domain = companies.domain.apply(lambda x: 0 if pd.isna(x) else 1)
companies.twitter_username = companies.twitter_username.apply(lambda x: 0 if pd.isna(x) else 1)
companies.homepage_url = companies.homepage_url.apply(lambda x: 0 if pd.isna(x) else 1)
companies.short_description = companies.short_description.apply(lambda x: 0 if pd.isna(x) else 1)
companies.description = companies.description.apply(lambda x: 0 if pd.isna(x) else 1)
companies.overview = companies.overview.apply(lambda x: 0 if pd.isna(x) else 1)
companies = companies.drop(['parent_id'], axis=1)

companies.head()

,id,entity_type,name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,...,funding_rounds,funding_total_usd,created_at,updated_at,product_count,closed_product_count,operating_product_count,office_count,closed_office_count,operating_office_count
0,c:1,Company,Wetpaint,web,operating,2005-10-17,NaN,1,1,1,...,3,39750000.0,2007-05-25 06:51:27,2013-04-13 03:29:00,2.0,0.0,2.0,2.0,0,2.0
1,c:10,Company,Flektor,games_video,acquired,NaN,NaN,1,1,1,...,0,0.0,2007-05-31 21:11:51,2008-05-23 23:23:14,1.0,0.0,1.0,1.0,0,1.0
2,c:100,Company,There,games_video,acquired,NaN,NaN,1,1,1,...,0,0.0,2007-08-06 23:52:45,2013-11-04 02:09:48,0.0,0.0,0.0,1.0,0,1.0
3,c:10000,Company,MYWEBBO,network_hosting,operating,2008-07-26,NaN,1,1,1,...,0,0.0,2008-08-24 16:51:57,2008-09-06 14:19:18,0.0,0.0,0.0,0.0,0,0.0
4,c:10001,Company,THE Movie Streamer,games_video,operating,2008-07-26,NaN,1,1,1,...,0,0.0,2008-08-24 17:10:34,2008-09-06 14:19:18,0.0,0.0,0.0,0.0,0,0.0


In [3]:
# print(companies.status.unique())
# companies.status.value_counts()
# companies.closed_at.count() # understanding that the acquired companies neeeded to be closed

companies_condition = (
    (companies['status'] == 'operating') &
    companies['category_code'].isna() & 
    companies['founded_at'].isna() & 
    (companies['domain'] == 0) &
    (companies['twitter_username']== 0) &
    (companies['homepage_url']== 0) &
    (companies['short_description']== 0) &
    (companies['description']== 0) &
    (companies['overview']== 0) &
    companies['tag_list'].isna() &
    (companies['region'] == 'unknown') &
    companies['last_investment_at'].isna() &
    (companies['investment_rounds'] == 0) & 
    (companies['invested_companies'] == 0) & 
    companies['last_funding_at'].isna() & 
    (companies['funding_rounds'] == 0) & 
    (companies['funding_total_usd'] == 0) &
    (companies.product_count <= companies.closed_product_count) &
    (companies.office_count <= companies.closed_office_count) 
)

# Updating the status of the companies that meet the conditions
companies.loc[companies_condition, 'status'] = 'closed' 
# companies_that_should_be_closed.describe()
companies.status.value_counts()


status
operating    123680
closed        62345
acquired       9394
ipo            1134
Name: count, dtype: int64

In [4]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196553 entries, 0 to 196552
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       196553 non-null  object 
 1   entity_type              196553 non-null  object 
 2   name                     196552 non-null  object 
 3   category_code            123186 non-null  object 
 4   status                   196553 non-null  object 
 5   founded_at               91227 non-null   object 
 6   closed_at                2620 non-null    object 
 7   domain                   196553 non-null  int64  
 8   homepage_url             196553 non-null  int64  
 9   twitter_username         196553 non-null  int64  
 10  short_description        196553 non-null  int64  
 11  description              196553 non-null  int64  
 12  overview                 196553 non-null  int64  
 13  tag_list                 81452 non-null   object 
 14  coun

In [17]:
# op_comp = companies[
#     (companies['status'] == 'operating') & 
#     companies['founded_at'].isna() & 
#     (companies['invested_companies'] == 0) & 
#     (companies['funding_rounds'] == 0) &
#     (companies['domain'] == 0) &
#     companies['category_code'].isna()
#     ]
# op_comp

,id,entity_type,name,category_code,status,founded_at,closed_at,domain,homepage_url,twitter_username,...,funding_rounds,funding_total_usd,created_at,updated_at,product_count,closed_product_count,operating_product_count,office_count,closed_office_count,operating_office_count


In [15]:
# Filter to include only 'operating' companies
companies['founded_at'] = pd.to_datetime(companies['founded_at'], errors='coerce')
companies['last_investment_at'] = pd.to_datetime(companies['last_investment_at'], errors='coerce')
companies['last_funding_at'] = pd.to_datetime(companies['last_funding_at'], errors='coerce')


# # print(companies['created_at'].dt.year)
# # op_comp = (companies['created_at'].dt.year <= 2008)

# # op_comp.head()

operating_companies_condition = (
    (companies['status'] == 'operating') &
    (companies['founded_at'].dt.year < 2008 | companies['founded_at'].isna()) &
    (companies['last_investment_at'].dt.year < 2008  | companies['last_investment_at'].isna()) & 
    (companies['last_funding_at'].dt.year < 2008  | companies['last_funding_at'].isna())
)

# Updating the status of the companies that meet the conditions
companies.loc[operating_companies_condition, 'status'] = 'closed' 
# companies_that_should_be_closed.describe()
companies.status.value_counts()

# print(len(old_companies))

# Updating the status of the companies that meet the conditions




status
operating    123680
closed        62345
acquired       9394
ipo            1134
Name: count, dtype: int64

In [ ]:
BEFORE // not altered dataset

status
operating    183441
acquired       9394
closed         2584
ipo            1134
Name: count, dtype: int64

AFTER // product count + office count + no data about creation, inv, funding.

status
operating    123680
closed        62345
acquired       9394
ipo            1134
Name: count, dtype: int64

In [1]:
companies.to_csv('./merged/new_companies.csv', index=False)

NameError: name 'companies' is not defined